#### Custom Tool Calling and ReAct Agent

**Approach**: Use agents that can select and use multiple tools:
1. **RAG Tool**: For questions about ingested documents
2. **Web Search Tool**: For real-time information and current events
3. **Custom Tools**: For domain-specific data (e.g., stock prices via APIs)

**Agent Capabilities**:
- **Tool Selection**: Automatically chooses the right tool(s) for each query
- **Multi-Step Reasoning**: Can use multiple tools in sequence
- **Context Awareness**: Understands when to use RAG vs. web search vs. custom tools

This creates a comprehensive system that handles both document-based and real-time queries.

In [1]:
# Install notebook dependencies. 
# Will take a while to download and install numerous dependencies. 
# Wait until it finishes before proceeding
%pip install llama_stack_client==0.2.22 yfinance

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.0 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 196.5 MB/s  0:00:00
  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15636 sha256=64c98e6bb41f7c52978842746abbf66673e16e6301170ba4366eb660c09031ae
  Stored in directory: /tmp/pip-ephem-whee

In [2]:
"""
Multi-Tool Agent: Yahoo Finance + Tavily Search
Intelligent routing between stock data and web search

This script uses ReActAgent with JSON response format and combines:
1. Yahoo Finance Tool - For Indian banks (HDFC, ICICI, SBI) stock data
2. Tavily Search - For all other real-time information
"""

from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.client_tool import client_tool
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
from llama_stack_client.types import UserMessage
from typing import cast, Iterator
import yfinance as yf

import os

#### We write a custom function that wraps the Yahoo Finance API and decorate with annotations to inform Llamastack that this should be treated as a tool.

The ReActAgent of Llamastack will check the query and call the appropriate tool

In [3]:
@client_tool
def indian_bank_stock(ticker: str, period: str = "current"):
    """
    Fetch stock price and financial data for Indian banks.
    
    :param ticker: Stock ticker symbol (HDFCBANK.NS, ICICIBANK.NS, or SBIN.NS)
    :param period: Time period ("current", "1mo", "3mo", "6mo", "1y")
    :return: Stock information with price, volume, market cap
    """
    print(f"\n[STOCK TOOL] Called with ticker={ticker}, period={period}")

    # We are only interested in our competitor bank stocks
    valid_tickers = ["HDFCBANK.NS", "ICICIBANK.NS", "SBIN.NS"]
    if ticker.upper() not in valid_tickers:
        print(f"[STOCK TOOL] Invalid ticker: {ticker}")
        return {"error": f"Only supports: {', '.join(valid_tickers)}", "ticker": ticker}
    
    try:
        stock = yf.Ticker(ticker)
        
        if period != "current":
            hist = stock.history(period=period)
            if hist.empty:
                return {"error": "No data", "ticker": ticker}
            
            first = hist['Close'].iloc[0]
            last = hist['Close'].iloc[-1]
            change_pct = ((last - first) / first) * 100
                        
            return {
                "ticker": ticker.upper(),
                "period": period,
                "start_price": float(first),
                "end_price": float(last),
                "change_percent": float(change_pct),
                "start_date": hist.index[0].strftime('%Y-%m-%d'),
                "end_date": hist.index[-1].strftime('%Y-%m-%d'),
            }
        
        # Current data
        info = stock.info
        price = info.get('currentPrice') or info.get('regularMarketPrice')
        prev = info.get('previousClose')
        
        print(f"[STOCK TOOL] Current price: ₹{price}")
        
        return {
            "ticker": ticker.upper(),
            "company": info.get('longName'),
            "current_price": float(price) if price else None,
            "previous_close": float(prev) if prev else None,
            "market_cap": info.get('marketCap'),
            "volume": info.get('volume'),
        }
        
    except Exception as e:
        print(f"[STOCK TOOL] Error: {e}")
        return {"error": str(e)}

In [4]:
# LlamaStack service URL (in-cluster)
LLAMASTACK_URL = "http://llama-stack-dist-service.competitor-analysis.svc.cluster.local:8321"

# Get Tavily search API key from environment variable
# Tavily is a search API that provides web search capabilities
tavily_search_api_key = os.getenv('TAVILY_SEARCH_API_KEY')

# Configure provider data for web search
# If API key is available, pass it to enable web search functionality
if tavily_search_api_key is None:
    provider_data = None  # Web search will not be available
else:
    # provider_data: Configuration for external service providers
    # tavily_search_api_key: API key for Tavily search service
    provider_data = {"tavily_search_api_key": tavily_search_api_key}

# Reinitialize client with provider data for web search
# provider_data enables the client to use external services like Tavily
client = LlamaStackClient(
    base_url=LLAMASTACK_URL,
    provider_data=provider_data  # Enables web search if API key is provided
)

models = client.models.list()
model_id = next(m.identifier for m in models if m.model_type == "llm")

INFO:httpx:HTTP Request: GET http://llama-stack-dist-service.competitor-analysis.svc.cluster.local:8321/v1/models "HTTP/1.1 200 OK"


In [5]:
# =============================================================================
# MULTI-TOOL REACTIVE AGENT
# =============================================================================

# Global agent instance (create once, reuse)
_agent = None
_session_id = None


def initialize_multi_tool_agent():
    """Initialize the agent with both tools (call once)."""
    global _agent, _session_id
    
    if _agent is not None:
        return _agent, _session_id
    
    # Initialize client
    client = LlamaStackClient(
        base_url=LLAMASTACK_URL,
        provider_data={
            "tavily_search_api_key": tavily_search_api_key,
        }
    )
    
    # Create ReActAgent with two specialized tools
    _agent = ReActAgent(
        client=client,
        model=model_id,
        tools=[indian_bank_stock, "builtin::websearch"],
        response_format={
            "type": "json_schema",
            "json_schema": ReActOutput.model_json_schema(),
        },
        sampling_params={
            "strategy": {"type": "greedy"},
            "max_tokens": 1512,
        },
    )
    
    _session_id = _agent.create_session("multi-tool-session")
    
    return _agent, _session_id


In [6]:
def run_multi_tool_query(query: str):
    """
    Run a query using the multi-tool agent.
    Agent intelligently routes to the appropriate tool.
    """
    
    # Get or create agent
    agent, session_id = initialize_multi_tool_agent()
    
    # Execute query
    print(f"User Query: {query}\n")
    print("Agent Response:\n")
    
    response = agent.create_turn(
        messages=[UserMessage(role="user", content=query)],
        session_id=session_id,
        stream=True,
    )
    
    # Log response
    try:
        for log in EventLogger().log(cast(Iterator, response)):
            log.print()
    except Exception as e:
        print(f"\n❌ ERROR during logging: {e}")
        import traceback
        traceback.print_exc()

In [7]:
# SHOULD USE indian_bank_stock custom tool
run_multi_tool_query("What's the current stock price of HDFC Bank?")

INFO:httpx:HTTP Request: GET http://llama-stack-dist-service.competitor-analysis.svc.cluster.local:8321/v1/tools?toolgroup_id=builtin%3A%3Awebsearch "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llama-stack-dist-service.competitor-analysis.svc.cluster.local:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://llama-stack-dist-service.competitor-analysis.svc.cluster.local:8321/v1/tools?toolgroup_id=builtin%3A%3Awebsearch "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llama-stack-dist-service.competitor-analysis.svc.cluster.local:8321/v1/agents/a093665a-5aef-4a52-bc73-b28b1d35b58d/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llama-stack-dist-service.competitor-analysis.svc.cluster.local:8321/v1/agents/a093665a-5aef-4a52-bc73-b28b1d35b58d/session/31a9933b-0f32-4e9b-b949-f3bab3f136a1/turn "HTTP/1.1 200 OK"


User Query: What's the current stock price of HDFC Bank?

Agent Response:

inference> {
    "action": {
        "tool_name": "indian_bank_stock",
        "tool_params": [
            {
                "name": "ticker",
                "value": "HDFCBANK.NS"
            }
        ]
    },
    "answer": null


,

"thought": "I need to fetch the current stock price of HDFC Bank. I will use the 'indian_bank_stock' tool for this purpose."
}

[STOCK TOOL] Called with ticker=HDFCBANK.NS, period=current


INFO:httpx:HTTP Request: POST http://llama-stack-dist-service.competitor-analysis.svc.cluster.local:8321/v1/agents/a093665a-5aef-4a52-bc73-b28b1d35b58d/session/31a9933b-0f32-4e9b-b949-f3bab3f136a1/turn/8e0b5566-71f6-4831-ab67-b0ab41915785/resume "HTTP/1.1 200 OK"


[STOCK TOOL] Current price: ₹1004.1
tool_execution> Tool:indian_bank_stock Response:{"ticker": "HDFCBANK.NS", "company": "HDFC Bank Limited", "current_price": 1004.1, "previous_close": 997.2, "market_cap": 15445539160064, "volume": 6741764}
inference> {
    "action": null,
    "thought": "I have obtained the current stock price of HDFC Bank. The current price is 1004.1.",
    "answer":  "The current stock price of HDFC Bank is $1004.10."
}


In [9]:
# SHOULD USE tavily websearch tool
run_multi_tool_query("What is the current exchange rate of USD to INR?")

INFO:httpx:HTTP Request: POST http://llama-stack-dist-service.competitor-analysis.svc.cluster.local:8321/v1/agents/a093665a-5aef-4a52-bc73-b28b1d35b58d/session/31a9933b-0f32-4e9b-b949-f3bab3f136a1/turn "HTTP/1.1 200 OK"


User Query: What is the current exchange rate of USD to INR?

Agent Response:

inference> {
    "action": {
        "tool_params": [
            {
                "name": "query",
                "value": "current USD to INR exchange rate"
            }
        ],
        "tool_name": "web_search"
    },
    "thought": "I need to find the current exchange rate of USD to INR. I will use the 'web_search' tool for this purpose.",
    "answer": null
}


INFO:httpx:HTTP Request: POST http://llama-stack-dist-service.competitor-analysis.svc.cluster.local:8321/v1/tool-runtime/invoke "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llama-stack-dist-service.competitor-analysis.svc.cluster.local:8321/v1/agents/a093665a-5aef-4a52-bc73-b28b1d35b58d/session/31a9933b-0f32-4e9b-b949-f3bab3f136a1/turn/7124a598-6c60-4dd0-b9e2-3efaabdfbb63/resume "HTTP/1.1 200 OK"


tool_execution> Tool:web_search Response:{"query": "current USD to INR exchange rate", "top_k": [{"url": "https://www.xe.com/en-us/currencyconverter/convert/?Amount=1&From=USD&To=INR", "title": "1 USD to INR - US Dollars to Indian Rupees Exchange Rate - Xe", "content": "1 USD equals 90.15 INR using the current mid-market exchange rate of \u20b990.1505. If you're looking to send 1 USD to INR, check if Xe could save you money on your", "score": 0.94109917, "raw_content": null}, {"url": "https://www.bookmyforex.com/currency-converter/usd-to-inr/", "title": "USD to INR | Convert US Dollar to Indian Rupee - BookMyForex", "content": "Our online currency converter is showing you the value of 1 US Dollar in Indian Rupees according to the current foreign exchange rate'of INR 89.975. Today i.e.", "score": 0.9363841, "raw_content": null}, {"url": "https://wise.com/us/currency-converter/usd-to-inr-rate?amount=1", "title": "1 US dollar to Indian rupees Exchange Rate. Convert USD/INR - Wise", "conte